In [1]:
import os
import glob
import pandas as pd

In [2]:
# Define working directory
os.chdir("C:/Users/samya/VS Code Projects/Python/Python Projects/Spotify_hit_predictor/backend")

In [3]:
ds00 = pd.read_csv('Original_data/dataset-of-00s.csv')
ds10 = pd.read_csv('Original_data/dataset-of-10s.csv')
ds60 = pd.read_csv('Original_data/dataset-of-60s.csv')
ds70 = pd.read_csv('Original_data/dataset-of-70s.csv')
ds80 = pd.read_csv('Original_data/dataset-of-80s.csv')
ds90 = pd.read_csv('Original_data/dataset-of-90s.csv')

merge = [ds00, ds10, ds60, ds70, ds80, ds90]
df = pd.concat(merge)
df.reset_index(drop=True, inplace=True)

In [4]:
df2 = df.drop(['track', 'artist', 'uri',"key", "tempo", "speechiness"], axis=1)

In [5]:
#Validation of correlation between variables
import numpy as np 
df3=df2.corr()

In [6]:
# Reformat data
data = df2.values
X = data[:, 0:12]  
y = data[:, 12]

print(data.shape, X.shape, y.shape)

(41106, 13) (41106, 12) (41106,)


In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=10)

## Data Preprocessing

In [8]:
#Scale features (X) using MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler(feature_range=(0,1)).fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test) 

X_train_scaled

array([[0.78947368, 0.52283395, 0.76172236, ..., 0.8       , 0.19905999,
        0.06508876],
       [0.54048583, 0.83094117, 0.77921392, ..., 0.8       , 0.21633373,
        0.04142012],
       [0.5       , 0.66588373, 0.7455139 , ..., 0.8       , 0.20480589,
        0.03550296],
       ...,
       [0.57692308, 0.36777999, 0.68303866, ..., 0.8       , 0.15844718,
        0.09467456],
       [0.41902834, 0.16070793, 0.50819858, ..., 0.8       , 0.200772  ,
        0.04733728],
       [0.29554656, 0.24873856, 0.67937808, ..., 0.6       , 0.12021911,
        0.06508876]])

One-hot encode the labels

In [9]:
#One-hot encode output labels (y)
from tensorflow.keras.utils import to_categorical
y_train_categorical = to_categorical(y_train)
y_test_categorical = to_categorical(y_test)

y_train_categorical

array([[0., 1.],
       [1., 0.],
       [0., 1.],
       ...,
       [0., 1.],
       [0., 1.],
       [1., 0.]], dtype=float32)

## Creating and defining our Deep Learning Model Architecture

In [10]:
#Create a sequential model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
model = Sequential() 

number_inputs = 12  

#Create hidden layers
model.add(Dense(units=14,activation='relu', input_dim=number_inputs))
model.add(Dense(units=120,activation='relu'))
model.add(Dense(units=80,activation='relu'))

#Create output layer
number_classes = 2
model.add(Dense(units=number_classes, activation='softmax')) 

Number of Hidden Nodes 
https://stats.stackexchange.com/questions/181/how-to-choose-the-number-of-hidden-layers-and-nodes-in-a-feedforward-neural-netw#:~:text=The%20number%20of%20hidden%20neurons,size%20of%20the%20input%20layer.

In [11]:
#Model Summary
model.summary() 

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 14)                182       
                                                                 
 dense_1 (Dense)             (None, 120)               1800      
                                                                 
 dense_2 (Dense)             (None, 80)                9680      
                                                                 
 dense_3 (Dense)             (None, 2)                 162       
                                                                 
Total params: 11,824
Trainable params: 11,824
Non-trainable params: 0
_________________________________________________________________


In [12]:
#Compile the Model
import tensorflow as tf

opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

## Training the Model

In [13]:
#Training the Model
history = model.fit(X_train_scaled,y_train_categorical,epochs=500, batch_size=2000, shuffle=True, verbose=2)

Epoch 1/500
16/16 - 0s - loss: 0.6684 - accuracy: 0.6322 - 391ms/epoch - 24ms/step
Epoch 2/500
16/16 - 0s - loss: 0.6087 - accuracy: 0.6816 - 52ms/epoch - 3ms/step
Epoch 3/500
16/16 - 0s - loss: 0.5678 - accuracy: 0.6952 - 41ms/epoch - 3ms/step
Epoch 4/500
16/16 - 0s - loss: 0.5500 - accuracy: 0.7105 - 50ms/epoch - 3ms/step
Epoch 5/500
16/16 - 0s - loss: 0.5370 - accuracy: 0.7250 - 50ms/epoch - 3ms/step
Epoch 6/500
16/16 - 0s - loss: 0.5272 - accuracy: 0.7324 - 60ms/epoch - 4ms/step
Epoch 7/500
16/16 - 0s - loss: 0.5203 - accuracy: 0.7396 - 40ms/epoch - 2ms/step
Epoch 8/500
16/16 - 0s - loss: 0.5158 - accuracy: 0.7416 - 50ms/epoch - 3ms/step
Epoch 9/500
16/16 - 0s - loss: 0.5119 - accuracy: 0.7442 - 51ms/epoch - 3ms/step
Epoch 10/500
16/16 - 0s - loss: 0.5089 - accuracy: 0.7453 - 49ms/epoch - 3ms/step
Epoch 11/500
16/16 - 0s - loss: 0.5072 - accuracy: 0.7468 - 50ms/epoch - 3ms/step
Epoch 12/500
16/16 - 0s - loss: 0.5064 - accuracy: 0.7494 - 80ms/epoch - 5ms/step
Epoch 13/500
16/16 - 0s

Epoch 101/500
16/16 - 0s - loss: 0.4803 - accuracy: 0.7660 - 42ms/epoch - 3ms/step
Epoch 102/500
16/16 - 0s - loss: 0.4806 - accuracy: 0.7655 - 42ms/epoch - 3ms/step
Epoch 103/500
16/16 - 0s - loss: 0.4806 - accuracy: 0.7657 - 47ms/epoch - 3ms/step
Epoch 104/500
16/16 - 0s - loss: 0.4807 - accuracy: 0.7646 - 34ms/epoch - 2ms/step
Epoch 105/500
16/16 - 0s - loss: 0.4814 - accuracy: 0.7649 - 44ms/epoch - 3ms/step
Epoch 106/500
16/16 - 0s - loss: 0.4803 - accuracy: 0.7649 - 37ms/epoch - 2ms/step
Epoch 107/500
16/16 - 0s - loss: 0.4802 - accuracy: 0.7644 - 34ms/epoch - 2ms/step
Epoch 108/500
16/16 - 0s - loss: 0.4793 - accuracy: 0.7659 - 42ms/epoch - 3ms/step
Epoch 109/500
16/16 - 0s - loss: 0.4797 - accuracy: 0.7671 - 42ms/epoch - 3ms/step
Epoch 110/500
16/16 - 0s - loss: 0.4810 - accuracy: 0.7651 - 41ms/epoch - 3ms/step
Epoch 111/500
16/16 - 0s - loss: 0.4810 - accuracy: 0.7656 - 42ms/epoch - 3ms/step
Epoch 112/500
16/16 - 0s - loss: 0.4798 - accuracy: 0.7660 - 42ms/epoch - 3ms/step
Epoc

Epoch 200/500
16/16 - 0s - loss: 0.4706 - accuracy: 0.7728 - 48ms/epoch - 3ms/step
Epoch 201/500
16/16 - 0s - loss: 0.4698 - accuracy: 0.7726 - 59ms/epoch - 4ms/step
Epoch 202/500
16/16 - 0s - loss: 0.4712 - accuracy: 0.7702 - 56ms/epoch - 3ms/step
Epoch 203/500
16/16 - 0s - loss: 0.4704 - accuracy: 0.7720 - 58ms/epoch - 4ms/step
Epoch 204/500
16/16 - 0s - loss: 0.4691 - accuracy: 0.7733 - 56ms/epoch - 3ms/step
Epoch 205/500
16/16 - 0s - loss: 0.4691 - accuracy: 0.7720 - 65ms/epoch - 4ms/step
Epoch 206/500
16/16 - 0s - loss: 0.4697 - accuracy: 0.7740 - 57ms/epoch - 4ms/step
Epoch 207/500
16/16 - 0s - loss: 0.4694 - accuracy: 0.7722 - 55ms/epoch - 3ms/step
Epoch 208/500
16/16 - 0s - loss: 0.4707 - accuracy: 0.7727 - 50ms/epoch - 3ms/step
Epoch 209/500
16/16 - 0s - loss: 0.4703 - accuracy: 0.7715 - 51ms/epoch - 3ms/step
Epoch 210/500
16/16 - 0s - loss: 0.4689 - accuracy: 0.7733 - 49ms/epoch - 3ms/step
Epoch 211/500
16/16 - 0s - loss: 0.4685 - accuracy: 0.7740 - 50ms/epoch - 3ms/step
Epoc

Epoch 299/500
16/16 - 0s - loss: 0.4615 - accuracy: 0.7774 - 43ms/epoch - 3ms/step
Epoch 300/500
16/16 - 0s - loss: 0.4607 - accuracy: 0.7778 - 43ms/epoch - 3ms/step
Epoch 301/500
16/16 - 0s - loss: 0.4607 - accuracy: 0.7791 - 49ms/epoch - 3ms/step
Epoch 302/500
16/16 - 0s - loss: 0.4606 - accuracy: 0.7787 - 50ms/epoch - 3ms/step
Epoch 303/500
16/16 - 0s - loss: 0.4614 - accuracy: 0.7776 - 50ms/epoch - 3ms/step
Epoch 304/500
16/16 - 0s - loss: 0.4607 - accuracy: 0.7783 - 42ms/epoch - 3ms/step
Epoch 305/500
16/16 - 0s - loss: 0.4600 - accuracy: 0.7779 - 34ms/epoch - 2ms/step
Epoch 306/500
16/16 - 0s - loss: 0.4606 - accuracy: 0.7778 - 51ms/epoch - 3ms/step
Epoch 307/500
16/16 - 0s - loss: 0.4615 - accuracy: 0.7768 - 49ms/epoch - 3ms/step
Epoch 308/500
16/16 - 0s - loss: 0.4609 - accuracy: 0.7773 - 42ms/epoch - 3ms/step
Epoch 309/500
16/16 - 0s - loss: 0.4600 - accuracy: 0.7783 - 51ms/epoch - 3ms/step
Epoch 310/500
16/16 - 0s - loss: 0.4605 - accuracy: 0.7777 - 50ms/epoch - 3ms/step
Epoc

Epoch 398/500
16/16 - 0s - loss: 0.4542 - accuracy: 0.7825 - 34ms/epoch - 2ms/step
Epoch 399/500
16/16 - 0s - loss: 0.4545 - accuracy: 0.7830 - 43ms/epoch - 3ms/step
Epoch 400/500
16/16 - 0s - loss: 0.4543 - accuracy: 0.7820 - 41ms/epoch - 3ms/step
Epoch 401/500
16/16 - 0s - loss: 0.4532 - accuracy: 0.7831 - 50ms/epoch - 3ms/step
Epoch 402/500
16/16 - 0s - loss: 0.4526 - accuracy: 0.7832 - 49ms/epoch - 3ms/step
Epoch 403/500
16/16 - 0s - loss: 0.4537 - accuracy: 0.7831 - 42ms/epoch - 3ms/step
Epoch 404/500
16/16 - 0s - loss: 0.4522 - accuracy: 0.7837 - 30ms/epoch - 2ms/step
Epoch 405/500
16/16 - 0s - loss: 0.4530 - accuracy: 0.7827 - 58ms/epoch - 4ms/step
Epoch 406/500
16/16 - 0s - loss: 0.4541 - accuracy: 0.7829 - 33ms/epoch - 2ms/step
Epoch 407/500
16/16 - 0s - loss: 0.4530 - accuracy: 0.7830 - 50ms/epoch - 3ms/step
Epoch 408/500
16/16 - 0s - loss: 0.4522 - accuracy: 0.7836 - 49ms/epoch - 3ms/step
Epoch 409/500
16/16 - 0s - loss: 0.4525 - accuracy: 0.7822 - 42ms/epoch - 3ms/step
Epoc

Epoch 497/500
16/16 - 0s - loss: 0.4470 - accuracy: 0.7854 - 36ms/epoch - 2ms/step
Epoch 498/500
16/16 - 0s - loss: 0.4475 - accuracy: 0.7859 - 43ms/epoch - 3ms/step
Epoch 499/500
16/16 - 0s - loss: 0.4472 - accuracy: 0.7863 - 50ms/epoch - 3ms/step
Epoch 500/500
16/16 - 0s - loss: 0.4471 - accuracy: 0.7863 - 51ms/epoch - 3ms/step


## Validation of the Model

In [14]:
#Evaluate the Model using the testing data
#Compare Model performace between training and testing data
model_loss_train, model_accuracy_train = model.evaluate(X_train_scaled, y_train_categorical, verbose=2)
model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test_categorical, verbose=2)

print(f"Train --> Loss: {model_loss_train}, Accuracy: {model_accuracy_train}")    
print(f"Test --> Loss: {model_loss}, Accuracy: {model_accuracy}")

964/964 - 1s - loss: 0.4460 - accuracy: 0.7871 - 799ms/epoch - 829us/step
322/322 - 0s - loss: 0.4716 - accuracy: 0.7753 - 198ms/epoch - 614us/step
Train --> Loss: 0.4460458755493164, Accuracy: 0.7870511412620544
Test --> Loss: 0.471639484167099, Accuracy: 0.7753235101699829


## Saving the Trained Model

In [15]:
# Save the model
model.save("../Models/h5/skey&tempo&speech.h5")